In [ ]:
# First training results

https://www.comet.ml/michaelbornholdt/deepprofiler-lincs/3751bddf2ea24527a7f617becbe89383

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pycytominer import aggregate, annotate, normalize, feature_select, cyto_utils
from cytominer_eval import evaluate
from cytominer_eval.transform import metric_melt
from sklearn.decomposition import PCA
import sys
#sys.path.insert(0, "/Users/mbornhol/git/neural-profiling")

In [9]:
df = pd.read_csv('727_aggregated_median.csv', low_memory=False)

In [10]:
df_meta = pd.read_csv('/Users/mbornhol/git/neural-profiling/pre-trained/data-prep/02_index_preperation/repurposing_info_external_moa_map_resolved.tsv', sep='\t')
df_meta = df_meta[['broad_sample', 'moa']]
df_meta.rename(columns = {'broad_sample' : 'Metadata_broad_sample', 'moa' : 'Metadata_moa'}, inplace=True)

In [11]:
features = [f for f in df.columns if f.startswith('eff')]
meta_features = [feat for feat in df.columns if feat not in features]
top_moa = df.Metadata_moa.value_counts()[1:20].keys().tolist()

In [12]:
k_range = [5, 10, 15, 20, 25]
p_range = np.arange(0.995, 0.96, -0.005)

In [14]:
df = df.rename(columns={'Metadata_Compound' : 'Metadata_broad_sample'})

In [15]:
sphere = normalize(
    profiles=df,
    features=features,
    samples="Metadata_broad_sample == 'DMSO'",
    method='spherize',
    output_file='none'
)

In [16]:
con = aggregate(
    sphere,
    strata=["Metadata_broad_sample"],
    features=features
)

In [17]:
con = pd.merge(con, df_meta, how='left', on=['Metadata_broad_sample'])
res_sphere = evaluate(
        profiles=con,
        features=features,
        meta_features=['Metadata_broad_sample', "Metadata_moa"],
        replicate_groups=["Metadata_moa"],
        operation="enrichment",
        similarity_metric="pearson",
        enrichment_percentile=p_range
    )
res_sphere

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.991040,0.000000,1.000000
1,0.990,0.989737,0.000000,1.000000
2,0.985,0.987814,0.000000,1.000000
3,0.980,0.985724,0.000000,1.000000
4,0.975,0.984200,1.872294,0.300357
5,0.970,0.982916,1.546271,0.381833
6,0.965,0.981070,1.314900,0.459379
7,0.960,0.978640,1.142186,0.531309


In [5]:
df.columns

Index(['Unnamed: 0', 'Metadata_Aggregate_On', 'Metadata_Well_Position',
       'Metadata_Plate', 'Metadata_Well', 'Metadata_moa',
       'Metadata_mmoles_per_liter', 'Metadata_dose_recode',
       'Metadata_Concentration', 'Metadata_Treatment_ID',
       ...
       'efficientnet_1270', 'efficientnet_1271', 'efficientnet_1272',
       'efficientnet_1273', 'efficientnet_1274', 'efficientnet_1275',
       'efficientnet_1276', 'efficientnet_1277', 'efficientnet_1278',
       'efficientnet_1279'],
      dtype='object', length=1296)

In [6]:
df = pd.merge(df, df_meta, how='left', right_on=['Metadata_broad_sample'], left_on=['Metadata_Compound'])

In [25]:
d = {}
for moa in df.Metadata_moa.unique():
    d[moa] = len(df[df['Metadata_moa'] == moa].Metadata_broad_sample.unique())

In [26]:
d

{'unknown': 1,
 'acetylcholine receptor antagonist': 3,
 'HMGCR inhibitor': 2,
 'HDAC inhibitor': 2,
 'bacterial DNA gyrase inhibitor': 1,
 'EGFR inhibitor|protein tyrosine kinase inhibitor': 1,
 'DNA methyltransferase inhibitor': 1,
 'fungal squalene epoxidase inhibitor': 1,
 'benzodiazepine receptor agonist': 1,
 'PKC activator': 1,
 'adrenergic receptor agonist': 1,
 'Aurora kinase inhibitor': 1,
 'gamma secretase inhibitor': 2,
 'glutamate receptor antagonist': 1,
 'CFTR channel agonist': 1,
 'calcium channel blocker': 1,
 'serotonin receptor agonist|serotonin receptor antagonist': 2,
 'EGFR inhibitor': 2,
 'hypoxia inducible factor inhibitor': 1,
 'DNA alkylating agent': 1,
 'sodium channel blocker': 1,
 'XIAP inhibitor': 2,
 'HSP inhibitor': 1,
 'AKT inhibitor': 2,
 'angiotensin converting enzyme inhibitor': 2,
 'leukotriene receptor antagonist': 1,
 '5 alpha reductase inhibitor': 1,
 'adrenergic receptor antagonist': 3,
 'c-Met inhibitor': 1,
 'vitamin D receptor agonist': 1,
 '

In [27]:
index = pd.read_csv('/Users/mbornhol/git/neural-profiling/pre-trained/data-prep/02_index_preperation/sub_index.csv')